In [2]:
import os
import math
import numpy as np
from PIL import Image
from pyquaternion import Quaternion
%matplotlib inline
from nuscenes.nuscenes import NuScenes
import mmengine
import os
from scipy.spatial import KDTree

from nuscenes.utils.data_classes import RadarPointCloud

nusc = NuScenes(version='v1.0-trainval', dataroot='/opt/zhh/nuscenes/', verbose=True)
camera_types = [
            'CAM_FRONT',
            'CAM_FRONT_RIGHT',
            'CAM_FRONT_LEFT',
            'CAM_BACK',
            'CAM_BACK_LEFT',
            'CAM_BACK_RIGHT',
        ]
RADARS_FOR_CAMERA = {
            'CAM_FRONT_LEFT': ["RADAR_FRONT_LEFT", "RADAR_FRONT"],
            'CAM_FRONT_RIGHT': ["RADAR_FRONT_RIGHT", "RADAR_FRONT"],
            'CAM_FRONT': ["RADAR_FRONT_RIGHT", "RADAR_FRONT_LEFT", "RADAR_FRONT"],
            'CAM_BACK_LEFT': ["RADAR_BACK_LEFT", "RADAR_FRONT_LEFT"],
            'CAM_BACK_RIGHT': ["RADAR_BACK_RIGHT", "RADAR_FRONT_RIGHT"],
            'CAM_BACK': ["RADAR_BACK_RIGHT", "RADAR_BACK_LEFT"]}
   
rad_root = "/opt/zhh/nuscenes/radar_map/"
num_sweeps=3
for sample in mmengine.track_iter_progress(nusc.sample):
    for cam in camera_types:
        radars_related =RADARS_FOR_CAMERA[cam]
        radar_tokens = [sample['data'][_] for _ in radars_related]
        radar_records = [nusc.get("sample_data", _) for _ in radar_tokens]
        camera_token = sample['data'][cam]
        camera_record = nusc.get("sample_data", camera_token)
        img_path=os.path.join(nusc.dataroot, camera_record['filename'])
        # im = Image.open(img_path)
        lidar_token = sample['data']['LIDAR_TOP']
        lidar_record = nusc.get("sample_data", lidar_token)
        # Get current point cloud
        point_cloud_list = []
        for i in range(len(radar_records)):
            # Original version: all radar points are mapped to LIDAR_TOP
            radar_record = radar_records[i]
            point_clouds, times = RadarPointCloud.from_file_multisweep(nusc, sample,
                                                                            radar_record["channel"], "LIDAR_TOP",
                                                                            nsweeps=num_sweeps)
            point_cloud_list.append(point_clouds.points)
        point_clouds = RadarPointCloud(np.concatenate(tuple(point_cloud_list), axis=1))

        #增加高度
        a=point_clouds.points
        a_1=np.tile(a,(1,5))
        a_1[2,(a.shape[1]):2*(a.shape[1])]+=0.2
        a_1[2,2*(a.shape[1]):3*(a.shape[1])]+=0.5
        a_1[2,3*(a.shape[1]):4*(a.shape[1])]+=-0.2
        a_1[2,4*(a.shape[1]):5*(a.shape[1])]+=1
        a_1[2,5*(a.shape[1]):6*(a.shape[1])]+=-0.3
        #加高开关
        point_clouds.points=a_1



        point_cloud_coords = point_clouds.points[:3, :].transpose()
        kdtree = KDTree(point_cloud_coords)
        filtered_point_indices = []
        for x in sample["anns"]:
            ann_record=nusc.get('sample_annotation',x)
            sample_record = nusc.get('sample', ann_record['sample_token'])
            lidar = sample_record['data']['LIDAR_TOP']
            data_path, grt_box, camera_intrinsic = nusc.get_sample_data(lidar, selected_anntokens=[x])
            query=grt_box[0].center
            width, length, height = grt_box[0].wlh
            r_max=np.sqrt((width/2)**2+(length/2)**2+(height/2)**2)
            indices = kdtree.query_ball_point(query,r_max+1)
            filtered_point_indices.extend(indices)
        #去除重复的点云索引
        filtered_point_indices = list(set(filtered_point_indices))
        # 提取过滤后的点云数据
        filtered_point_cloud = point_clouds.points[:,filtered_point_indices]
        #过滤开关
        point_clouds.points=filtered_point_cloud




        # First step: transform the point-cloud to the ego vehicle frame for the timestamp of the sweep.
        cs_record = nusc.get('calibrated_sensor', lidar_record['calibrated_sensor_token'])
        point_clouds.rotate(Quaternion(cs_record['rotation']).rotation_matrix)
        point_clouds.translate(np.array(cs_record['translation']))
        # Second step: transform to the global frame.
        poserecord = nusc.get('ego_pose', lidar_record['ego_pose_token'])
        point_clouds.rotate(Quaternion(poserecord['rotation']).rotation_matrix)
        point_clouds.translate(np.array(poserecord['translation']))
        # Third step: transform into the ego vehicle frame for the timestamp of the image.
        poserecord = nusc.get('ego_pose', camera_record['ego_pose_token'])
        point_clouds.translate(-np.array(poserecord['translation']))
        point_clouds.rotate(Quaternion(poserecord['rotation']).rotation_matrix.T)
        # Fourth step: transform into the camera.
        cs_record = nusc.get('calibrated_sensor', camera_record['calibrated_sensor_token'])
        point_clouds.translate(-np.array(cs_record['translation']))
        point_clouds.rotate(Quaternion(cs_record['rotation']).rotation_matrix.T)
        # Fifth step: actually take a "picture" of the point cloud.
                # Grab the depths (camera frame z axis points away from the camera).

        #为了便于后面投影图像后拼接特征，先把点特征搞出来，所以只提【5，8，9】即rcs,补偿速度
        points_feature=point_clouds.points[[0,1,2,5,8,9],:]


        filename, ext = os.path.splitext(img_path)
        point_path=filename + '.npy'
        path_parts = point_path.split('/')
        rpath_new= rad_root + '/'.join(path_parts[4:])
        np.save(rpath_new, points_feature)




Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 100.816 seconds.
Reverse indexing ...
Done reverse indexing in 8.0 seconds.
[>>>>>>>>>>>>>>>>>>>>>>>] 34149/34149, 2.9 task/s, elapsed: 11897s, ETA:     0s
